In [226]:
from numpy import *
from pylab import *
from math import erf
import pandas as pd

In [227]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


D:\Program Files (x86)\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['fft', 'power', 'random', 'linalg']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [228]:
# constant cell

r = 0.07
sigma = 0.09
k = 120

N = 70
s_start = 0
s_end = 490
s = linspace(s_start, s_end, N + 1)
ds = abs(s_end - s_start)/N  # ds, uniform gri

M = 100
m = linspace(1,M+1, M+1)
t_start = 0
t_end = 1  # T
time = 0
t = linspace(t_start, t_end, M + 1)
dt = abs(t_end - t_start)/M  # dt, uniform grid

In [229]:
# function cell for Euler's method

V = [0 for i in range(N + 1)]  # exact solution V(s, T-t)
W = [0 for i in range(N + 1)]  # numerical solution W(s, T-t)
s_second_partial = [0 for i in range(N + 1)]
s_first_partial = [0 for i in range(N + 1)]
time_error = [0 for i in range(M + 1)]
log_E = [0 for i in range(M + 1)]

In [230]:
def aprx_func(W):
    for i in range(1,N):
        s_second_partial[i] = (W[i+1] - 2*W[i] + W[i-1])/ds**2 # midpoint formula for second order
        s_first_partial[i] = (W[i+1] - W[i-1])/(2*ds) # midpoint formula for first order
    
    return 1/2 * sigma**2 * (s*s)*s_second_partial + r*s*s_first_partial - r*float_(W)

def normal_dist(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def CDF(t, idx):
    arr = [0 for i in range(N + 1)]
    for i in range(1, N+1):
        if idx == 1:
            arr[i] = normal_dist((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
        else:
            arr[i] = normal_dist((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            #sym.erf((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))
            
    return arr

def exact_func(t):
    return s*float_(CDF(t,1)) - k*exp(-r*t)*float_(CDF(t,2))

In [231]:
time = 0
#clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
        V[i] = W[i]
    else:
        W[i] = 0
        V[i] = W[i]
        
#plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
#plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
#title('W(x) at time = ' + str(round(time, 2)))
#legend()
#axis((90, 150, 0, 45))
#pause(1)

In [232]:
# RK2 for midpoint method
clf()
i = 1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    time_error[M-i] = sum(abs(V-W))
    i = i+1
    #clf()
   # plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
    #plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
    #title('W(x) at time = ' + str(round(time, 2)))
    #axis((90, 150, 0, 45))
    #legend()
    #pause(0.001)
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK2')
legend()

In [233]:
# RK3
i=1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt * RK1)
    RK3 = aprx_func(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    time_error[M-i] = sum(abs(V-W))
    i = i+1
    
    #clf()
    #plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
    #plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
    #title('W(x) at time = ' + str(round(time, 2)))
    #axis((90, 150, 0, 45))
    #legend()
    #pause(0.001)
    
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK3')
legend()

In [235]:
# RK4
i=1
while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    RK3 = aprx_func(W + dt/2 * RK2)
    RK4 = aprx_func(W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    time_error[M-i] = sum(abs(V-W))
    i = i+1
    #time_error[i] = sum(abs(V-W))
    #i = i+1
    
    #clf()
    #plot(s,W,'c', linestyle = '--',linewidth = 5, label = 'numerical')
    #plot(s,V,'r', linestyle = ':', linewidth = 3, label = 'exact')
    #title('W(x) at time = ' + str(round(time, 2)))
    #axis((90, 150, 0, 45))
    #legend()
    #pause(0.001)
plot(t,time_error,linestyle='--',linewidth = 1.8, label='RK4')
legend()

In [234]:
# AB3 -> 왜 돼?

while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt * RK1)
    RK3 = aprx_func(W + dt/2 * (RK1 + RK2)/2)
    
    
    F = aprx_func(W)
    F[0] = 0
    for i in range(1, 3):
        F[i] = W[i] + dt/6 * (RK1[i] + RK2[i] + 4*RK3[i])
        
    for i in range(2, N+1):
        W[i] = W[i] + dt/12 * (23*F[i] - 16*F[i-1] + 5*F[i-2])
    
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    clf()
    plot(s, W,'-b')
    title('W(x) at time = ' + str(round(time, 2)))
    axis((70, 130, 0, 50))
    pause(0.001)

In [212]:
# AB2 -> 왜 돼?

while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    
    F = aprx_func(W)
    F[0] = 0
    for i in range(1, 2):
        F[i] = W[i] + dt*RK2[i]
        
    for i in range(2, N+1):
        W[i] = W[i] + dt/2 * (3*F[i] - 1*F[i-1])
    
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    clf()
    plot(s, W,'-b')
    title('W(x) at time = ' + str(round(time, 2)))
    axis((70, 130, 0, 50))
    pause(0.001)

In [274]:
# PC3 - > 잘 안됨

while time < t_end:
    time = time + dt
    
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt * RK1)
    RK3 = aprx_func(W + dt/2 * (RK1 + RK2)/2)
    
    F = aprx_func(W)
    F[0] = 0
    W_ = W
    for i in range(1, 3):
        W[i] = W[i] + dt/6 * (RK1[i] + RK2[i] + 4*RK3[i])
        W_[i] = W[i]
    
            
    
    for i in range(2, N+1):
        W_[i] = W[i] + dt/12 * (23*F[i] - 16*F[i-1] + 5*F[i-2])
    
    F_ = aprx_func(W_)
    for i in range(1, N+1):
        W[i] = W[i] + dt/12 * (5*F_[i] + 8*F[i] - F[i-1])
    
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    clf()
    plot(s, W,'-b')
    title('W(x) at time = ' + str(round(time, 2)))
    axis((70, 130, 0, 50))
    pause(0.001)